# Scrape data from [Schweizer Bauern](https://www.hofsuche.schweizerbauern.ch)

## filter data
We're only interested in farms from zurich, so we filter the data accordingly (``extSearch.query.q=Zürich``): https://www.hofsuche.schweizerbauern.ch/de?extSearch.query.q=Zürich&project=vomhof&extSearch=true

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep

options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)

farm_data = []

try:
    driver.get("https://www.hofsuche.schweizerbauern.ch/de?extSearch.query.q=Zürich&project=vomhof&extSearch=true")
    sleep(5)

    # Click the "Mehr laden" button until it's no longer available or a set number of times to load all content
    while True:
        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(2)  # wait for the page to scroll down and content to load

        try:
            load_more_button = driver.find_element(By.XPATH, '//button[contains(., "Mehr laden")]')
            load_more_button.click()
            sleep(4)  # wait for the content to load after clicking
        except Exception as e:
            print("All content loaded or button not found.")
            break

    # Extract the required data
    farm_cards = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/de/farm/"]')
    for card in farm_cards:
        href = card.get_attribute("href")
        title = card.find_element(By.CSS_SELECTOR, "div.card.de").get_attribute("title")
        farm_data.append({"title": title, "url": href})

finally:
    driver.quit()

print(farm_data)

In [ ]:
import csv

with open("../data/raw_data/farm_overview.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "URL"])

    for farm in farm_data:
        writer.writerow([farm["title"], farm["url"]])